In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_ta as ta # pip install pandas-ta
import yfinance as yf # pip install yfinance

In [3]:
listings = ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIA']

In [21]:
df = yf.download(tickers=listings, period='10y', interval='1d', auto_adjust=False)

[*********************100%***********************]  31 of 31 completed


In [22]:
df.describe()

Adj Close                                                      \
              AAPL         AMGN          AXP           BA          CAT   
count  2519.000000  2519.000000  2519.000000  2519.000000  2519.000000   
mean     66.379830   164.739335   100.731453   198.432366   122.105867   
std      51.741239    50.402766    36.505742    88.042334    55.316957   
min      12.209949    72.422432    45.823326    82.948372    47.813442   
25%      25.649494   125.770210    71.717438   122.283867    72.341423   
50%      41.650196   156.729065    90.485977   172.837936   114.604889   
75%     119.454876   212.952301   120.587765   247.087349   164.105682   
max     180.949997   285.160858   194.931152   430.299957   263.111816   

                                                                        ...  \
               CRM         CSCO          CVX          DIA          DIS  ...   
count  2519.000000  2519.000000  2519.000000  2519.000000  2519.000000  ...   
mean    130.867618    34.128616    94.957955   222.364892   110.184505  ...   
std      66.130128    12.221318    28.102112    72.817287    30.451335  ...   
min      36.750000    15.029694    46.736900   118.474129    55.172844  ...   
25%      72.494999    22.156258    77.813442   151.068596    93.137634  ...   
50%     128.089996    36.495636    87.693855   222.506912   103.145279  ...   
75%     174.565002    43.866463   100.093174   285.686508   124.975002  ...   
max     309.959991    61.063244   184.339294   357.968903   201.910004  ...   

             Volume                                                          \
                MRK          MSFT           NKE            PG           TRV   
count  2.519000e+03  2.519000e+03  2.519000e+03  2.519000e+03  2.519000e+03   
mean   1.117509e+07  3.141332e+07  7.673381e+06  8.370618e+06  1.641265e+06   
std    5.642430e+06  1.599784e+07  4.241646e+06  5.370043e+06  7.374572e+05   
min    2.343223e+06  7.425600e+06  1.821900e+06  2.022100e+06  3.054000e+05   
25%    7.960896e+06  2.227650e+07  5.288150e+06  6.033100e+06  1.155050e+06   
50%    9.965851e+06  2.774550e+07  6.683400e+06  7.292100e+06  1.460700e+06   
75%    1.279042e+07  3.566750e+07  8.629100e+06  9.255450e+06  1.916100e+06   
max    1.024973e+08  2.484285e+08  5.731940e+07  1.237357e+08  6.498300e+06   

                                                                             
                UNH             V            VZ           WBA           WMT  
count  2.519000e+03  2.519000e+03  2.519000e+03  2.519000e+03  2.519000e+03  
mean   3.689434e+06  9.035617e+06  1.703452e+07  5.965792e+06  8.089301e+06  
std    1.885419e+06  4.928527e+06  1.465702e+07  3.635321e+06  4.601942e+06  
min    7.140000e+05  1.993500e+06  4.108300e+06  1.367000e+06  2.094900e+06  
25%    2.609800e+06  6.205400e+06  1.172575e+07  3.955150e+06  5.644450e+06  
50%    3.239500e+06  7.937000e+06  1.492710e+07  5.186200e+06  6.918100e+06  
75%    4.251550e+06  1.050045e+07  1.941030e+07  6.943000e+06  9.146400e+06  
max    2.736140e+07  9.007360e+07  6.166205e+08  8.413010e+07  8.089810e+07  

[8 rows x 186 columns]

In [24]:
# stack and unstack indexed columns
df_unstacked = df.stack(level=0).unstack(level=1)

# set each column name to TICKER - VALUE
df_unstacked.columns = df_unstacked.columns.map(lambda x: f'{x[0]} - {x[1]}')

df_unstacked.head()

,AAPL - Adj Close,AAPL - Close,AAPL - High,AAPL - Low,AAPL - Open,AAPL - Volume,AMGN - Adj Close,AMGN - Close,AMGN - High,AMGN - Low,...,WBA - High,WBA - Low,WBA - Open,WBA - Volume,WMT - Adj Close,WMT - Close,WMT - High,WMT - Low,WMT - Open,WMT - Volume
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-03,13.975490,16.097143,16.155714,15.802857,16.097500,372352400.0,75.890427,99.129997,101.250000,97.220001,...,48.529999,46.560001,47.720001,6661600.0,60.895191,75.690002,75.800003,74.639999,75.059998,8440500.0
2013-06-04,13.931771,16.046785,16.229643,15.978214,16.186428,292728800.0,74.481804,97.290001,100.360001,97.120003,...,48.950001,47.950001,48.270000,6093400.0,61.096325,75.940002,76.379997,75.470001,75.769997,9344000.0
2013-06-05,13.801543,15.896786,16.097143,15.846786,15.916071,290589600.0,72.422432,94.599998,98.330002,94.570000,...,49.310001,48.580002,48.680000,6227400.0,60.541195,75.250000,75.949997,75.070000,75.720001,6778600.0
2013-06-06,13.595345,15.659286,15.964286,15.501786,15.909643,416934000.0,74.634926,97.489998,97.529999,94.150002,...,48.970001,47.639999,48.639999,7411900.0,60.846920,75.629997,75.739998,74.839996,75.269997,9425100.0
2013-06-07,13.699223,15.778929,15.830000,15.456071,15.589286,404535600.0,75.645462,98.809998,100.260002,98.320000,...,49.509998,48.549999,48.680000,4596100.0,61.410091,76.330002,76.870003,76.190002,76.379997,8347200.0


In [31]:
df_unstacked.to_csv('djia_data.csv', index=True)

In [36]:
df = pd.read_csv('djia_data.csv')

In [37]:
df.describe()

,AAPL - Adj Close,AAPL - Close,AAPL - High,AAPL - Low,AAPL - Open,AAPL - Volume,AMGN - Adj Close,AMGN - Close,AMGN - High,AMGN - Low,...,WBA - High,WBA - Low,WBA - Open,WBA - Volume,WMT - Adj Close,WMT - Close,WMT - High,WMT - Low,WMT - Open,WMT - Volume
count,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2.519000e+03,2519.000000,2519.000000,2519.000000,2519.000000,...,2519.000000,2519.000000,2519.000000,2.519000e+03,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2.519000e+03
mean,66.379830,68.252573,68.966358,67.469845,68.196290,1.532596e+08,164.739335,187.291398,189.170004,185.331786,...,62.668162,61.347634,62.023513,5.965792e+06,93.550393,101.374546,102.169694,100.583724,101.345196,8.089301e+06
std,51.741239,51.251540,51.851680,50.581953,51.193665,9.918225e+07,50.402766,44.044927,44.469534,43.615921,...,16.984590,16.815978,16.902648,3.635321e+06,32.475396,29.351792,29.662098,29.065062,29.360068,4.601942e+06
min,12.209949,14.063571,14.295357,13.888214,13.977143,3.519590e+07,72.422432,94.599998,96.500000,94.150002,...,29.959999,29.480000,29.750000,1.367000e+06,47.991093,56.419998,57.060001,56.299999,56.389999,2.094900e+06
25%,25.649494,28.061250,28.309999,27.743750,28.041250,8.746540e+07,125.770210,157.115005,158.814995,155.099998,...,48.810001,47.535000,48.270000,3.955150e+06,63.083426,75.519997,75.930000,75.084999,75.505001,5.644450e+06
50%,41.650196,43.654999,43.855000,43.299999,43.637501,1.223452e+08,156.729065,182.550003,184.500000,180.520004,...,62.279999,60.889999,61.450001,5.186200e+06,86.023712,93.110001,93.720001,92.250000,93.209999,6.918100e+06
75%,119.454876,121.145000,123.105000,119.635002,121.145000,1.877628e+08,212.952301,227.925003,230.409996,226.040001,...,79.500000,78.000000,78.780003,6.943000e+06,128.140228,132.004997,133.040001,130.680000,131.730003,9.146400e+06
max,180.949997,182.009995,182.940002,179.259995,182.630005,1.065523e+09,285.160858,292.390015,296.670013,289.260010,...,97.300003,96.290001,97.029999,8.413010e+07,156.764130,159.869995,160.770004,159.070007,160.250000,8.089810e+07
